In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import time
from konlpy.tag import Okt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences  
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from hanspell import spell_checker
from json.decoder import JSONDecodeError


In [ ]:
# 웹페이지로 리포트 보기
# pr=data.profile_report() # 프로파일링 결과 리포트를 pr에 저장
# data.profile_report() # 바로 결과 보기

In [ ]:
# CSV 파일이 있는 상위 폴더 경로
folder_path = "data"

# 상위 폴더 내의 모든 파일 목록 가져오기
file_names = os.listdir(folder_path)

# CSV 파일 목록만 추려내기
csv_files = [file_name for file_name in file_names if file_name.endswith(".csv")]

# 데이터프레임을 저장할 리스트 초기화
df_list = []

# CSV 파일 불러와서 데이터프레임 생성 후 리스트에 추가
for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path, sep="|", quoting=3, encoding="utf-8")
    df_list.append(df)

# 모든 데이터프레임을 하나로 병합
merged_df = pd.concat(df_list, axis=0, ignore_index=True)

# 병합한 데이터프레임을 하나의 CSV 파일로 저장
merged_df.to_csv("merged_data.csv", index=False, sep="|", encoding="utf-8")

# 결과 출력
print("모든 CSV 파일을 병합하여 merged_data.csv 파일로 저장")


In [ ]:
# CSV 파일 불러오기
csv_file_path = "C:/python/language_analysis/merged_data.csv"

df = pd.read_csv(csv_file_path, sep="|", encoding="utf-8")

In [ ]:
print('모든 데이터프레임의 길이 합:', len(df))

In [ ]:
# 열의 중복 제거
df.drop_duplicates(subset=['reviews'], inplace=True)

In [ ]:
print('모든 데이터프레임의 길이 합:', len(df))

In [ ]:
# 리뷰에 포함된 줄 바꿈 html 제거
df['reviews'] = df['reviews'].str.replace('<br/>', ' ')
df['reviews'] = df['reviews'].str.replace('.', '')

# 점수에 포함된 html 제거
df['star'] = df['star'].str.replace('<div class="num">', '')
df['star'] = df['star'].str.replace('</div>', '')

# 정규표현식 한글만 남김
df['reviews'] = df['reviews'].apply(lambda x: re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", str(x)))

In [ ]:
# Null 값이 존재하는지 확인
print(df.isnull().sum())

In [ ]:
# Null 값이 존재하는 행 제거
df = df.dropna(how = 'any')

# Null 값이 존재하는지 확인
print(df.isnull().values.any())

In [ ]:
# Null 값이 존재하는지 확인
print(df.isnull().sum())

In [ ]:
# 별점이 숫자형이 아니기 때문에 실수형으로 변환
df['star'] = df['star'].astype(float)

# 7.6 같은 별점의 경우 정수형으로 변환 시 7로 변환
df['star'] = df['star'].astype(int) 

In [ ]:
# 정규표현식과 간단한 전처리를 끝낸 후 새롭게 저장
df.to_csv("new_merged_data.csv", index=False, sep="|", encoding="utf-8")

In [ ]:
# CSV 파일 불러오기
csv_file_path = "C:/python/language_analysis/new_merged_data.csv"

new_df = pd.read_csv(csv_file_path, sep="|", encoding="utf-8")

In [ ]:
# 원본 CSV 파일 경로
csv_file_path = "C:/python/language_analysis/new_merged_data.csv"

# CSV 파일 불러오기
df = pd.read_csv(csv_file_path, sep="|", encoding="utf-8")

# 데이터프레임을 10만 행씩 나누어 리스트에 저장
dfs_to_save = [df[i:i+100000] for i in range(0, len(df), 100000)]

# 10만 행씩 나눈 데이터프레임을 CSV 파일로 저장
# 데이터가 너무 커서 나눠서 실행 하기 위함
for i, df_chunk in enumerate(dfs_to_save):
    df_chunk.to_csv(f"chunk_{i+1}.csv", sep="|", encoding="utf-8", index=False)

In [ ]:
dfs = []

# 파일 경로를 생성하는데 사용할 패턴 정의
file_pattern = "C:/python/language_analysis/chunk_{}.csv"

# 1부터 4까지의 파일을 읽어와서 리스트에 추가
for i in range(1, 5):
    file_path = file_pattern.format(i)
    df = pd.read_csv(file_path, sep="|", encoding="utf-8")
    dfs.append(df)

# dfs 리스트에 있는 모든 데이터프레임을 순서대로 df1, df2, df3, df4에 할당
df1, df2, df3, df4 = dfs

In [ ]:
# 에러가 발생한 인덱스를 저장할 리스트
error_indices = []

# 몇 번 까지 실행이 완료 했는지 보기 위함
aa = 0
# 맞춤법 검사

# 데이터 크기가 커서 df1 ~ df4로 나눠서 작업 실행
# 총 소요시간 630분 -> 약 12시간 30분
for i in range(len(df4)):
    try:
        a = df4['reviews'][i]
        spelled_sent = spell_checker.check(a)
        hanspell_sent = spelled_sent.checked
        df4['reviews'][i] = hanspell_sent

        # i가 5000의 배수일 때 10분 동안 대기
        if i % 5000 == 0 and i > 0:
            print(f"Pausing for 10 minutes at index {i}")
            time.sleep(10)
            
    except JSONDecodeError as e:
        error_indices.append(i)  # 에러가 발생한 인덱스를 저장
        print(f"Error at index {i}: {e}")
    except Exception as e:
        print(f"An error occurred at index {i}: {e}")

    # 몇 번 까지 실행이 완료 했는지 보기 위함
    aa+=1
    print(aa)

print("Completed with errors at indices:", error_indices)

In [ ]:
# 가공이 끝난 데이터프레임을 finish_chunk_1.csv 파일로 저장
df1.to_csv("finish_chunk_1.csv", sep="|", encoding="utf-8", index=False)

In [ ]:
# 가공이 끝난 데이터프레임을 finish_chunk_1.csv 파일로 저장
df2.to_csv("finish_chunk_2.csv", sep="|", encoding="utf-8", index=False)

In [ ]:
# 가공이 끝난 데이터프레임을 finish_chunk_1.csv 파일로 저장
df3.to_csv("finish_chunk_3.csv", sep="|", encoding="utf-8", index=False)

In [ ]:
# 가공이 끝난 데이터프레임을 finish_chunk_1.csv 파일로 저장
df4.to_csv("finish_chunk_4.csv", sep="|", encoding="utf-8", index=False)

In [ ]:
for i in range(1, 5):    
    # 파일 저장하기
    df.to_csv("finish_chunk_".format(i), sep="|", encoding="utf-8", index=False)

In [ ]:
# CSV 파일이 있는 상위 폴더 경로
folder_path = "finish_data"

# 상위 폴더 내의 모든 파일 목록 가져오기
file_names = os.listdir(folder_path)

# CSV 파일 목록만 추려내기
csv_files = [file_name for file_name in file_names if file_name.endswith(".csv")]

# 데이터프레임을 저장할 리스트 초기화
df_list = []

# CSV 파일 불러와서 데이터프레임 생성 후 리스트에 추가
for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path, sep="|", quoting=3, encoding="utf-8")
    df_list.append(df)

# 모든 데이터프레임을 하나로 병합
merged_df = pd.concat(df_list, axis=0, ignore_index=True)

# 병합한 데이터프레임을 하나의 CSV 파일로 저장
merged_df.to_csv("merged_finish_data.csv", index=False, sep="|", encoding="utf-8")

# 결과 출력
print("모든 CSV 파일을 병합하여 merged_finish_data.csv 파일로 저장")


In [ ]:
df = pd.read_csv("merged_finish_data.csv", sep="|", quoting=3, encoding="utf-8")

In [ ]:
len(df)

In [ ]:
# Null 값이 존재하는지 확인
print(df.isnull().sum())

In [ ]:
# Null 값이 존재하는 행 제거
df = df.dropna(how = 'any')

# Null 값이 존재하는지 확인
print(df.isnull().values.any())

In [ ]:
# Null 값이 존재하는지 확인
print(df.isnull().sum())

In [ ]:
print('데이터프레임의 길이:', len(df))

In [ ]:
df.drop_duplicates(subset=['reviews'], inplace=True)

In [ ]:
# df를 무작위로 섞기
df_shuffled = df.sample(frac=1, random_state=42)

# train과 test에 7:3으로 나누기
train_data, test_data = train_test_split(df_shuffled, test_size=0.3, random_state=42)

# 각각을 CSV 파일로 저장
train_data.to_csv('train_data.csv', index=False, sep="|", encoding="utf-8")
test_data.to_csv('test_data.csv', index=False, sep="|", encoding="utf-8")


In [ ]:
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '분들', '이다']
# ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', '분들', "이다"]
'하다', '게', '분들', '는', '가', '를', '인', '과', '좀', '도', '에', '고', '지', '임', '네', '이다', '은', '들', '한', '나', '다', '의', '걍'

okt = Okt()
 
X_train = []
for sentence in tqdm(train_data['review']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

In [ ]:
X_test = []
# 데이터프레임 합친 것
for sentence in tqdm(test_data['review']):
    # 토큰화
    tokenized_sentence = okt.morphs(sentence, stem=True)
    # 불용어 제거
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]
    # 한글에서 자음 또는 모음만으로 이루어진 문자 삭제
    stopwords_removed_sentence = [word for word in stopwords_removed_sentence if not re.match(r'^[ㄱ-ㅎㅏ-ㅣ]+$', word)]
    
    X_test.append(stopwords_removed_sentence)

In [ ]:
# 기계가 텍스트를 숫자로 처리할 수 있도록 훈련 데이터와 테스트 데이터에 정수 인코딩을 수행해야 한다.
# 단어 집합이 생성되는 동시에 각 단어에 고유한 정수가 부여
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

print(tokenizer.word_index)

In [ ]:
threshold = 3

# 단어의 수
total_cnt = len(tokenizer.word_index)

# 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
rare_cnt = 0

# 훈련 데이터의 전체 단어 빈도수 총 합
total_freq = 0

# 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합
rare_freq = 0

# 단어와 빈도수의 쌍(pair)을 key와 value로 받기
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

In [ ]:
# 케라스 토크나이저의 인자로 넘겨주고 텍스트 시퀀스를 정수 시퀀스로 변환.
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print(X_train[:3])

In [ ]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

In [ ]:
# 패딩 작업
print('리뷰의 최대 길이 :',max(len(review) for review in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(review) for review in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
# 전체 샘플 중 길이가 max_len 이하인 샘플의 비율이 몇 %인지 확인하는 함수
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [ ]:
# max_len의 크기와 비율은 비례
max_len = 60
below_threshold_len(max_len, X_train)

In [ ]:
# 모든 샘플의 길이 max_len으로 맞춤
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
# 하이퍼파라미터인 임베딩 벡터의 차원 크기
embedding_dim = 100
# 은닉 상태의 크기
hidden_units = 128
# 모델 학습
for _ in range(5):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))
    # 다 대 일 구조
    # 마지막 시점에서 두 개의 선택지 중 하나를 예측하는 이진 분류 문제를 수행하는 모델
    model.add(LSTM(hidden_units))
    # 이진 분류 문제의 경우, 출력층에 로지스틱 회귀를 사용해야 하므로 활성화 함수로는 시그모이드 함수를 사용
    model.add(Dense(1, activation='sigmoid'))

    # 검증 데이터 손실(val_loss)이 증가하면, 과적합 징후므로 검증 데이터 손실이 4회 증가하면
    #  정해진 에포크가 도달하지 못하였더라도 학습을 조기 종료(Early Stopping)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
    # 사용하여 검증 데이터의 정확도(val_acc)가 이전보다 좋아질 경우에만 모델을 저장
    mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    # validation_split=0.2을 사용하여 훈련 데이터의 20%를 검증 데이터로 분리해서 사용하고,
    # 검증 데이터를 통해서 훈련이 적절히 되고 있는지 확인
    history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

In [ ]:
# 테스트 모델 저장
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [ ]:
with open('tokenizer.pickle', 'wb') as handle:
     pickle.dump(tokenizer, handle)

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [ ]:
sentiment_predict("전체적으로 무난 하였으나 방 크기가 작아서 조금 아쉬웠습니다")

In [ ]:
sentiment_predict("방이 너무 더러워서 환불 요청했지만 받아주지않고 너무 별로네요")

In [ ]:
sentiment_predict("다신 안감 서비스 최악")

In [ ]:
sentiment_predict("방 냄새남")

In [ ]:
sentiment_predict("와인잔이 대여라고 되어있어서 와인만 사서 갔는데..와인잔은 대여되지않고 판매입니다 참고하세요카운터에 문의했더니 다음날 지배인오면 알아보고 연락준다고 했는데 연락도 안옴.")

In [ ]:
sentiment_predict("너무 오래되고 냄새 너무남")

In [ ]:
sentiment_predict("저렴하고 깨끗하여 전반적으로 만족합니다다만 방음이 잘 안되는 단점이 있어요저는 소음이 있어도 잘 잤지만 예민하신 분은 힘들 수도 있을 것 같아요")

In [ ]:
sentiment_predict("더블 데이트 때 잘 이용하고 가요 너무 친절하고 좋았습니당")

In [ ]:
sentiment_predict("늦은퇴실로 예약했는데 갑자기 12시전에 전화오셔서 퇴실해달라고 했습니다, 잘못예약했나싶어 나오긴했지만  다시보니 늦은퇴실로 예약로 했었네요 ..돈 더 주고 예약한건데 그런 부주위한 점이 아쉽네요")

In [ ]:
sentiment_predict("생각보다 너무 좁고 화장실 슬리퍼는 뜯어질려고 해서… 춥지는 않았습니다.")

In [ ]:
sentiment_predict("친절하시지만 방에 담배 냄새는 어쩔수없네요...")

In [ ]:
sentiment_predict("불친절하고 방에 담배 냄새는 어쩔수없네요...")

In [ ]:
sentiment_predict("친절한건지 불친절한건지 모르겠고 방에 담배 냄새는 어쩔수없네요...")

In [ ]:
sentiment_predict("방에 담배 냄새는 어쩔수없네요...")

In [ ]:
sentiment_predict("수건도 더 챙겨주시고 물도 더 주셨어요 근데 빗에 머리카락이 그대로 뭉쳐져 있더라구요 그 부분은 조금 아쉬웠어요")

In [ ]:
sentiment_predict("비오는날이라 그런건지 아님 원래 그런건지 테이블 청소도 너무 안돼있고,크로아상 옆쪽에 하루살이들이 엄청 많더라구요 ..... 위생이 아쉽네요")

In [ ]:
sentiment_predict("연박인데..청소를 안해주시네요ㅜㅜ")

In [ ]:
sentiment_predict("종이가 맞는건지 아니면 어플이 맞는건지.. 라운지 이용도 7시부터라고 적어두셨는데 예약을 해야 가능한거 같더라고요 사용 가능하냐 물어봤더니 개인적으로는 불가능하다 하시고 한 한 두시간뒤에 보니까 파티하고 계시더라구여 안에서 세탁은 자유롭게 이용 가능했고 수압이 되게 약해요ㅠ 그래도 뜨거운물은 잘 나옵니다 방음은 전혀 안되고 옆집 윗집 앞집 소리 다 나요...아침에 청소하실때 그 소리도 다 들림 드르륵 거리는 소리")

In [ ]:
sentiment_predict("3박 4일 동안 너무 편안하고,, 깔끔하게 사용했어요^^ 아침 조식도 너무 든든하고 맛있었답니당 눈 뜨면 바로 광안리 바다가 눈 앞에 보인게 제일 좋았어용")

In [ ]:
sentiment_predict("위치도 좋고 했지만 예전에 비해 숙소 청결도가 좀 낮아졌어요. 화장실에서 냄새도 나고 방충망도 뜯겨져있고")

In [ ]:
sentiment_predict("이상한 벌레가 물린거빼곤 좋았어요")